In [1]:
from os import listdir
from os.path import isfile, join
from lxml import etree

from PIL import Image
from PIL import ImageOps as IO

In [2]:
# Define data directories
my_data = "../data"
annotations = my_data + '/Annotation'        # BO file to be merged with additional information
images = my_data + '/Images' # SAP file with order fulfilment dates
cropped = my_data + '/Cropped'

In [3]:
# Convert all pictures into cropped and autocontrasted version
# Explore all picture folders
folders = [f for f in listdir(annotations) if (~isfile(join(annotations, f)) &
                                                f.lower().startswith('n02'))]

for folder in folders:
    print(folder)
    current_folder = join(annotations, folder)
    files = [f for f in listdir(current_folder) if (isfile(join(current_folder, f)))]
    for file in files:
        # Read each picture
        current_file = join(current_folder, file)      
        picture_file = join(images, folder, file + '.jpg')
        img = Image.open(picture_file) 
        
        # Get crop region for each picture
        tree = etree.parse(current_file)
        for coord in tree.xpath("/annotation/object/bndbox"):
            xmin_el = int(coord.xpath("xmin")[0].text)
            xmax_el = int(coord.xpath("xmax")[0].text)
            ymin_el = int(coord.xpath("ymin")[0].text)
            ymax_el = int(coord.xpath("ymax")[0].text)
        
        img_cropped = img.crop((xmin_el,ymin_el,xmax_el,ymax_el))
        try:
            img_cropped = IO.grayscale(img_cropped)
            img_cropped = IO.equalize(img_cropped)
            # img_cropped = IO.autocontrast(img_cropped, 1)
        except 'OSError':
            pass
       
        out_picture_file = join(cropped, folder, file + '.jpg')
        img_cropped.save(out_picture_file)


n02085620-Chihuahua
n02085782-Japanese_spaniel
n02085936-Maltese_dog
n02086079-Pekinese
n02086240-Shih-Tzu
n02086646-Blenheim_spaniel
n02086910-papillon
n02087046-toy_terrier
n02087394-Rhodesian_ridgeback
n02088094-Afghan_hound
n02088238-basset
n02088364-beagle
n02088466-bloodhound
n02088632-bluetick
n02089078-black-and-tan_coonhound
n02089867-Walker_hound
n02089973-English_foxhound
n02090379-redbone
n02090622-borzoi
n02090721-Irish_wolfhound
n02091032-Italian_greyhound
n02091134-whippet
n02091244-Ibizan_hound
n02091467-Norwegian_elkhound
n02091635-otterhound
n02091831-Saluki
n02092002-Scottish_deerhound
n02092339-Weimaraner
n02093256-Staffordshire_bullterrier
n02093428-American_Staffordshire_terrier
n02093647-Bedlington_terrier
n02093754-Border_terrier
n02093859-Kerry_blue_terrier
n02093991-Irish_terrier
n02094114-Norfolk_terrier
n02094258-Norwich_terrier
n02094433-Yorkshire_terrier
n02095314-wire-haired_fox_terrier
n02095570-Lakeland_terrier
n02095889-Sealyham_terrier
n02096051-Aired

In [1]:
import cv2
print (cv2.__version__)

ImportError: DLL load failed: Le module spécifié est introuvable.